In [1]:
import pandas as pd

from pandarallel import pandarallel

from freeholdforecast.tasks.etl_ml_task import ETL_ML_Task

pd.options.display.max_columns = 100

task = ETL_ML_Task()


2022-09-02 23:34:34 INFO ETL_ML_Task Initialized task for ohio-clermont with run date 2016-09-01


In [2]:
task._get_df_raw()
task.df_raw


2022-09-02 23:34:34 INFO ETL_ML_Task Retrieving data
2022-09-02 23:34:34 INFO ETL_ML_Task Loading existing data
2022-09-02 23:34:34 INFO ETL_ML_Task Total parcels: 79338
2022-09-02 23:34:34 INFO ETL_ML_Task Total sales: 201733
2022-09-02 23:34:34 INFO ETL_ML_Task Successfully retrieved data


,Parid,Year of Sale,Month of Sale,Day of Sale,last_sale_date,Owner Name 1,House #,Street Name,Street Suffix,Valid Sale,Sale Price,Building Value,Land Value,Tax District,Property Class
0,012002A024.,1994,9,12,1994-09-12,SHELTON HOWARD D &,4100.0,None,STATE ROUTE 222,0,128000.0,155800,68700,1,R
1,012002A024.,2014,7,3,2014-07-03,PHILLIPS ERIN R,4100.0,None,STATE ROUTE 222,0,235000.0,155800,68700,1,R
2,012002A025.,2004,9,17,2004-09-17,HUMBERT EDWARD W & CAROL A,4104.0,DR,OLD SOUTH RIVERSIDE,8,100000.0,101300,83400,1,R
3,012002A025.,2011,12,6,2011-12-06,HUMBERT EDWARD A &,4104.0,DR,OLD SOUTH RIVERSIDE,8,188000.0,101300,83400,1,R
4,012002A027.,1991,11,25,1991-11-25,None,NaN,DR,OLD SOUTH RIVERSIDE,8,36300.0,7300,11200,1,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201728,539006.070.,2017,8,1,2017-08-01,KENNETH A & SHARON A CHAMBERS,NaN,ST,SMITH,1,250000.0,0,6600,53,R
201729,539006.071.,2017,8,1,2017-08-01,KENNETH A & SHARON A CHAMBERS,658.0,ST,SMITH,1,250000.0,225000,18600,53,R
201730,550229.005.,1986,3,1,1986-03-01,None,NaN,LN,CURLISS,8,128000.0,0,1471300,55,A
201731,550229.005.,2007,6,25,2007-06-25,CHARLES J KUBICKI LLC,NaN,LN,CURLISS,8,5015000.0,0,1471300,55,A


In [3]:
task._get_df_ready()
task.df_ready


2022-09-02 23:34:34 INFO ETL_ML_Task Preparing data
2022-09-02 23:34:34 INFO ETL_ML_Task Loading existing data
2022-09-02 23:34:36 INFO ETL_ML_Task Successfully prepared data


,transfer_in_6_months,transfer_in_12_months,transfer_in_24_months,date,year,month,dates_since_last_sale,Parid,Year of Sale,Month of Sale,Day of Sale,last_sale_date,Owner Name 1,House #,Street Name,Street Suffix,Valid Sale,Sale Price,Building Value,Land Value,Tax District,Property Class
0,0,0,0,2003-01-01,2003,1,0,414814A011.,2003,1,24,2003-01-24,HANDLEY MICHAEL H,1104.0,DR,GLENDALE,0,101000.0,101000,25000,41,R
1,0,0,0,2003-02-01,2003,2,1,414814A011.,2003,1,24,2003-01-24,HANDLEY MICHAEL H,1104.0,DR,GLENDALE,0,101000.0,101000,25000,41,R
2,0,0,0,2003-03-01,2003,3,2,414814A011.,2003,1,24,2003-01-24,HANDLEY MICHAEL H,1104.0,DR,GLENDALE,0,101000.0,101000,25000,41,R
3,0,0,0,2003-04-01,2003,4,3,414814A011.,2003,1,24,2003-01-24,HANDLEY MICHAEL H,1104.0,DR,GLENDALE,0,101000.0,101000,25000,41,R
4,0,0,0,2003-05-01,2003,5,4,414814A011.,2003,1,24,2003-01-24,HANDLEY MICHAEL H,1104.0,DR,GLENDALE,0,101000.0,101000,25000,41,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128075,0,0,0,2021-12-01,2021,12,127,414244.139.,2003,10,31,2003-10-31,ELDER STANLEY C & ALLISON N,4326.0,DR,BEECHMONT,0,107500.0,126700,20000,41,R
2128076,0,0,0,2022-01-01,2022,1,128,414244.139.,2003,10,31,2003-10-31,ELDER STANLEY C & ALLISON N,4326.0,DR,BEECHMONT,0,107500.0,126700,20000,41,R
2128077,0,0,0,2022-03-01,2022,3,129,414244.139.,2003,10,31,2003-10-31,ELDER STANLEY C & ALLISON N,4326.0,DR,BEECHMONT,0,107500.0,126700,20000,41,R
2128078,0,0,0,2022-05-01,2022,5,130,414244.139.,2003,10,31,2003-10-31,ELDER STANLEY C & ALLISON N,4326.0,DR,BEECHMONT,0,107500.0,126700,20000,41,R


In [4]:
task._get_df_encoded()


2022-09-02 23:34:36 INFO ETL_ML_Task Encoding data
2022-09-02 23:34:36 INFO ETL_ML_Task Train dates: 2006-08-01 to 2016-08-01
2022-09-02 23:34:36 INFO ETL_ML_Task Test dates: 2016-09-01 to 2018-09-01


2022-09-02 23:34:41 INFO ETL_ML_Task Successfully encoded data


In [5]:
# task._train_model("transfer_in_6_months")


In [6]:
task._train_model("transfer_in_12_months")


2022-09-02 23:34:41 INFO ETL_ML_Task Training model for transfer_in_12_months
2022-09-02 23:34:41 INFO ETL_ML_Task Train labels: 39565/786276 (5.03%)
2022-09-02 23:34:41 INFO ETL_ML_Task Train res labels: 39565/237390 (16.67%)
2022-09-02 23:34:41 INFO ETL_ML_Task Test labels: 11067/180297 (6.14%)
2022-09-02 23:34:41 INFO ETL_ML_Task Training minutes per model task: 30
2022-09-02 23:34:41 INFO ETL_ML_Task Memory GB per model task: 8
2022-09-02 23:34:41 INFO ETL_ML_Task Fitting model
[WARNING] [2022-09-02 23:34:44,604:Client-AutoML(1):ohio-clermont] Unknown dtype for X: int64, assuming it takes 8 bit/number
[WARNING] [2022-09-02 23:34:45,290:Client-AutoML(1):ohio-clermont] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (1799.415812)
[WARNING] [2022-09-02 23:34:45,290:Client-AutoML(1):ohio-clermont] Capping the per_run_time_limit to 899.0 to have time for a least 2 models in each process.
2022-09-03 00:04:46 

/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


2022-09-03 00:20:34 INFO ETL_ML_Task auto-sklearn results:
  Dataset name: ohio-clermont
  Metric: roc_auc
  Best validation score: 0.996757
  Number of target algorithm runs: 122
  Number of successful target algorithm runs: 107
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 15
  Number of target algorithms that exceeded the memory limit: 0

2022-09-03 00:21:24 INFO ETL_ML_Task Pred labels: 2120/180297 (1.18%)
2022-09-03 00:21:24 INFO ETL_ML_Task Precision: 0.83
2022-09-03 00:21:24 INFO ETL_ML_Task ROC AUC: 0.58
2022-09-03 00:21:24 INFO ETL_ML_Task Classification report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97    169230
           1       0.83      0.16      0.27     11067

    accuracy                           0.95    180297
   macro avg       0.89      0.58      0.62    180297
weighted avg       0.94      0.95      0.93    180297

2022-09-03 00:21:24 INFO ETL_ML_Task

In [7]:
task._train_model("transfer_in_24_months")


2022-09-03 00:21:26 INFO ETL_ML_Task Training model for transfer_in_24_months
2022-09-03 00:21:28 INFO ETL_ML_Task Train labels: 75824/786276 (9.64%)
2022-09-03 00:21:28 INFO ETL_ML_Task Train res labels: 75824/454944 (16.67%)
2022-09-03 00:21:28 INFO ETL_ML_Task Test labels: 20911/180297 (11.60%)
2022-09-03 00:21:28 INFO ETL_ML_Task Training minutes per model task: 30
2022-09-03 00:21:28 INFO ETL_ML_Task Memory GB per model task: 8
2022-09-03 00:21:28 INFO ETL_ML_Task Fitting model
[WARNING] [2022-09-03 00:21:29,348:Client-AutoML(1):ohio-clermont] Unknown dtype for X: int64, assuming it takes 8 bit/number
[WARNING] [2022-09-03 00:21:30,157:Client-AutoML(1):ohio-clermont] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (1799.279074)
[WARNING] [2022-09-03 00:21:30,157:Client-AutoML(1):ohio-clermont] Capping the per_run_time_limit to 899.0 to have time for a least 2 models in each process.
2022-09-03 00:51:29